In [ ]:
# default_exp main

# Main

> The main FastAPI app.

In [ ]:
#export
from fastapi import FastAPI, Header, HTTPException
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from typing import List, Dict
from datetime import datetime
from ast import literal_eval
from pathlib import Path
import os, json

In [ ]:
#export
def get_valid_api_keys():
    "Return a list of keys that can be used to authorise users of this API"
    keys = os.environ.get('X_API_KEYS')
    assert keys is not None, 'Environment variable X_API_KEYS not set'
    return literal_eval(keys)

In [ ]:
#export
def api_key_check(x_api_key):
    if x_api_key not in get_valid_api_keys():
        raise HTTPException(status_code=401, detail='Invalid x-api-key')

In [ ]:
#export
class TranslatePayload(BaseModel):
    source_language: str
    target_language: str
    source_texts: List[str]

In [ ]:
#export
class Translator:
    def translate(self, source_texts, source_language, target_language='en'):
        print('TODO: remove old translation code')
        return {}

In [ ]:
#export
def create_app():
    app = FastAPI()
    app.mount('/static', StaticFiles(directory='static'), name='static')
    translator = Translator()

    @app.get('/api-key-check')
    def _api_key_check(x_api_key: str = Header(None)):
        "Validates the `x-api-key` header parameter"
        api_key_check(x_api_key)
        return {'x-api-key': x_api_key, 'result': 'OK'}
    
    @app.options('/translate/')
    def _translate_options(x_api_key: str = Header(None)):
        "Validate endpoint before invoking the actual POST method"
#         api_key_check(x_api_key)

    @app.post('/translate/', response_model=Dict[str, str])
    def _translate(payload: TranslatePayload, x_api_key: str = Header(None)):
        "TODO"
#         api_key_check(x_api_key)
        print('x_api_key', x_api_key)
        print(payload)
        # TODO: return probabilities + top n?
        return translator.translate(payload.source_texts, payload.source_language, payload.target_language)
        # TODO: use proper model to translate - no cache yet, just keep it simple
#         return {k:fr_to_en.get(k, '') for k in payload.source_texts}
    
    return app

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script('10_main.ipynb')

Converted 10_main.ipynb.
